# # GPU Setting
---

In [ ]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# # Instant-NGP
---

In [ ]:
# #@title install dependencies
# ! git clone https://github.com/ashawkey/stable-dreamfusion.git

# %cd stable-dreamfusion

# # fix the commit 
# ! git reset --hard e496e35df6e9b67784c3b24504cd61a2e7c19a0d

# # install requirements
! pip install -r requirements.txt
! pip install git+https://github.com/NVlabs/nvdiffrast/

# install CUDA extensions (takes about 8 minutes!)
! pip install ./raymarching
! pip install ./shencoder
! pip install ./freqencoder
! pip install ./gridencoder

## # Training & Testing
* The default 5000 training steps take around 30 minutes to finish. A larger `Training_iters` usually leads to better results.
* If CUDA OOM, try to decrease `Max_steps` and `Training_nerf_resolution`.
* If the NeRF fails to learn anything (empty scene, only background), try to decrease `Lambda_entropy` which regularizes the learned opacity.

In [ ]:
#@markdown ####**Training Settings:**
Prompt_text = '(masterpiece:2.0), spreaded trump card on the table' #@param {type: 'string'}
Training_iters = 10000 #@param {type: 'integer'}
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
# CUDA_ray = True #@param {type: 'boolean'}
# View_dependent_prompt = True #@param {type: 'boolean'}
# FP16 = True #@param {type: 'boolean'}
Seed = 0 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Max_steps = 512 #@param {type: 'number'}
Checkpoint = 'latest' #@param {type: 'string'}

#@markdown ---

#@markdown ####**Output Settings:**
Workspace = "trial/card" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

# processings
Prompt_text = '"' + Prompt_text + '"'


#@title start training
import torch
torch.cuda.empty_cache()

# !python main.py \
#         -O --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} \
#         --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} \
#         --ckpt {Checkpoint} --save_mesh --max_steps {Max_steps}
        # --progressive_view
!python main.py -O --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} --ckpt {Checkpoint} --save_mesh --max_steps {Max_steps}

In [ ]:
#@title testing 
%run main.py -O --test --text {Prompt_text} --workspace {Workspace} --save_mesh

In [ ]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)

# # Vanilla NeRF
---

In [ ]:
!nvidia-smi

In [ ]:
# #@title install dependencies
# ! git clone https://github.com/ashawkey/stable-dreamfusion.git

# %cd stable-dreamfusion

# install requirements
! pip install -r requirements.txt
! pip install git+https://github.com/NVlabs/nvdiffrast/

In [ ]:
#@markdown ####**Training Settings:**
Prompt_text = "DSLR photo of panda" #@param {type: 'string'}
Training_iters = 3000 #@param {type: 'integer'}
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
# CUDA_ray = True #@param {type: 'boolean'}
# View_dependent_prompt = True #@param {type: 'boolean'}
# FP16 = True #@param {type: 'boolean'}
Seed = 0 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Num_steps = 64 #@param {type: 'number'}
Upsample_steps = 32 #@param {type: 'number'}
Checkpoint = 'latest' #@param {type: 'string'}

#@markdown ---

#@markdown ####**Output Settings:**
Workspace = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

# processings
Prompt_text = "'" + Prompt_text + "'"

In [ ]:
#@title start training
import torch
torch.cuda.empty_cache()

%run main.py -O --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} --ckpt {Checkpoint} --save_mesh --num_steps {Num_steps} --upsample_steps {Upsample_steps}

In [ ]:
#@title testing 
import torch
torch.cuda.empty_cache()

checkpoint = '/data/kimgh/stable-dreamfusion/trial/checkpoints/df_ep0021.pth'

%run main.py -O2 --test --workspace {Workspace} --save_mesh --ckpt {Checkpoint} --max_ray_batch 1024

In [ ]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")
 
rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)